# Лабораторная работа 4. Решающие деревья и леса. Кластеризация и понижение размерности.

## Решающие деревья и леса

Давайте рассмотрим следующую задачу: зная информацию о пациенте попробуем предсказать есть ли у него сердечное заболевание. Каждый пациент будет описываться 13 признаками, включающими в себя возраст, пол, информацию из электрокардиограммы и т.д.

Загрузите данные из файла *heart.dat.txt*, используя пандас, в переменную *data* (разделитель полей — пробел). Значение целевой переменной находится в последней колонке (*Absence_(1)_or_presence_(2)_of_heart_disease*). 

In [ ]:
data = 

Выделите признаки в переменную *X*, а целевую зависимость — в *y*.

In [ ]:
X, y = 

В начале рассмотрим решающее дерево. Как известно, одно дерево обладает плохой обобщающей сопособностью, поэтому стоит подбирать параметры, такие как высота дерева и минимальное число объектов в листе, для уменьшения этого явления.

Для оценки качества будем использовать 5-Fold кросс-валидацию с метрикой *AUC-ROC*. Создайте объект [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html) со следующими параметрами:
 - первым параметром нужно передать количество объектов в выборке
 - *n_folds* равно 5
 - установить *shuffle=True*, чтобы объекты были перемешаны перед разбиением
 - зафиксировать *random_state*

In [ ]:
kfold = 

Теперь воспользуемся функцией [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.cross_val_score.html) для оценки качества. Она принимает следующие параметры:
 - *estimator* — в данном случае им будет решающее дерево [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Здесь и далее фиксируйте параметр *random_state* у решающего дерева, чтобы результат был воспроизводим. В этом пункте ограничения на другие параметры не накладываются
 - *X* и *y* — матрица "объект-признак" и значение целевой переменной соответственно
 - *scoring* просто указать *roc_auc_score*
 - *cv* — передать созданный ранее kfold
 
Эта функция возвращает массив значений качества на каждом разбиении. 

**(1 балл)** Посчитайте среднее, минимальное и максимальное качество, которое получатеся с помощью решающего дерева.

**(0.5 балла)** Теперь попробуем посмотреть изменится ли оно, если ограничить высоту дерева. Зафиксируйте высоту дерева равной 3 и повторите аналогичные действия. Увеличились ли показания качества по сравнению с прошлым разом?

**(0.5 балла)** А принесет ли улучшение, если ограничить высоту дерева 1? 

**(1 балл)** Напишите почему наложение ограничения на высоту дерева влияет на его обобщающую способность. Почему деревья глубиной 1 по одиночке могут быть неэффективны?

**(2 балла)** Теперь попробуем провести аналогичные эксперименты для случая решающего леса [RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). Рассмотрите 10, 50 и 100 деревьев в случайном лесу (поле *n_estimators*) для высоты деревьев по умолчанию, 3 и 1. Не забудьте зафиксировать *random_state*. Улучшается ли качество при увеличении количества деревьев в лесу? Сильно ли влияет ограничение на высоту деревьев при построении леса?

**(1 балл)** И напоследок посмотрим, какие признаки являются наиболее *важными*. Обучите решающий лес с оптимальными на ваш взгляд параметрами (можете опираться на эксперименты ранее) на всей выборке и посмотрите на поле *feature\_importances\_* у обученного леса. Выпишите названия наиболее и наименее важных признаков с точки зрения случайного леса. 

## Кластеризация

В этой части работы вы будете работать с уже известным вам набором данных изображений цифр, но в данном случае мы рассмотрим задачу кластеризации. Целью работы будет посмотреть можно ли разделить изображения цифр по классам,  не зная их истинных меток.

Для начала вам потребуется загрузить набор данных, используя функцию [load_digits](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html), в переменную data:

In [ ]:
data = 

Сами изображения чисел в виде векторов хранятся в поле *data*, а истинные метки — в поле *target*. Сохраните изображения в переменную *X*, а истинные метки — *y*.

In [ ]:
X = 
y = 

Задача кластеризации состоит в следующем: нужно построить разбиение всех объектов на *K* кластеров, где *K* задано заранее. В данном случае в датасете присутствуют цифры от 0 до 9, поэтому *K* будет равно 10.

Для того, чтобы в дальнейшем оценивать качество получившейся кластеризации, будем использовать следующие метрики:
 - [Homogenity](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html) — эта метрика основывается на использовании информации об истинных метках объектов. Она позволяет оценить, все ли объекты в кластере имеют одну и ту же метку. В качестве первого параметра ей передаются истинные метки класса, второй параметр — предсказанные. Обратите внимание, что данная метрика не является симметричной. Данная метрика принимает значения от 0 до 1, где 1 соответствует наилучшей кластеризации.
 - [Silhouette Coefficient](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) — эта метрика, в отличии от предыдущей, оперирует только с исходной матрицей "объект-признак" и предсказанными метками. Она позволяет получить информацию насколько "хорошей" получилась кластеризация с точки зрения расположения объектов в кластере, оценивая насколько далеко они находятся друг от друга. Данная метрика принимает значения от -1 до 1, где 1 соответствует лучшей кластеризации, 0 говорит о том, что есть перекрывающиеся кластера.

**(1 балл)** Построение кластеризации будем производить с помощью метода [К-средних](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). Для этого вам нужно сделать следующее:
 - создать объект *KMeans*, имеющий следующие интересные для нас параметры:
   - количество кластеров *n_clusters*
   - *random_state* — его следует зафиксировать для воспроизводимости результата
 - вызвать метод *fit_predict*, передав на вход переменную *X* и сохранив результаты в переменную *predicted_labels*

In [ ]:
predicted_labels = 

**(2 балла)** Теперь вычислите значения указанных выше метрик для полученной кластеризации. Хорошее ли качество имеет данная кластеризация, исходя из интерпретации метрик?

**(1 балл)** Иногда полезно бывает посмотреть на результаты кластеризации. Поэтому воспользуемся методами уменьшения размерности, чтобы визуализировать результаты кластеризации. Для этих целей будем использовать два метода [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), который основан на получении новых признаков с помощью линейной комбинации старых, а так же [tSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), который преобразует пространство более сложным образом, пытаясь оставить рядом объекты, которые были близки в исходном пространстве. Эти методы имеют схожий интерфейс, поэтому в обоих случаях последовательность дейтствий будет заключаться в следующем:
 - создать соответствующий объект, указав количество компонент *n_components* равным 2
 - вызвать метод *fit_transform*, передав переменную *X* и сохранив результат в переменные *X_pca* и *X_tsne* соответственно
 
Обратите внимание, что преобразование *tSNE* может вычисляться некоторое время (несколько минут). 

In [ ]:
X_pca = 
X_tsne = 

**(1 балл)** Теперь выполните 2 кластеризации, но в качестве матрицы "объекты-признаки" передавайте полученные ранее *X_pca* и *X_tsne*. Сохраните результаты предсказания в переменные *predicted_labels_pca* и *predicted_labels_tsne* соответственно. Посчитайте качество каждой кластеризации с помощью указанных ранее метрик качества.

**(1 балл)** Сильно ли отличается качество новых кластеризаций от исходной для каждого способа сжатия размерности? Нашелся ли способ уменьшения размерности, который позволяет достичь наилучшего качества по обеим метрикам?

С помощью функции *plot_embedding* ниже можно посмотреть на визуализацию кластеризации. На вход ей передается двумерная матрица "обекты-признаки", истинные метки и предсказанные метки. Она строит на плоскости объекты, при этом числом обозначена истинная метка объекта, а одинаковым цветом — объекты одного кластера.

In [40]:
def plot_embedding(X, y, predicted_labels, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=(10, 10))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(predicted_labels[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()

**(1 балл)** Для каждой из матриц *X_pca* и *X_tsne* и результатов соответствующих кластеризаций (*predicted_labels_pca* и *predicted_labels_tsne*) вызовите функцию *plot_embedding*.

**(2 балла)** Проанализируйте полученные результаты:
 - какой из трех методов кластеризации: на исходных признаках, на признаках, полученных с помощью PCA или tSNE дал наилучший результат по обеим метрикам?
 - какой из методов сжатия размерности дает наилучшую визуализацию на плоскости?
 - можно ли сказать, что результаты визуализации согласуются с метриками: если кластеры кажутся отделимыми визуально, то и значения метрик выше?
 - как вы думаете: если увеличить число кластеров (то есть взять не 10, а 12, 15 и пр.) может ли кластеризация быть более качественной? Почему?